# Link

- https://medium.com/spidernitt/how-to-build-a-text-summarizer-from-scratch-1a68e39558c4
- https://arxiv.org/abs/1707.02268v3
- https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
- https://blog.floydhub.com/gentle-introduction-to-text-summarization-in-machine-learning/
- https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
- https://www.machinelearningman.com/post/building-a-text-summarizer-from-scratch
- https://outline.com/TGzqsF


# Arabic Text Summarizertion

# [1] Include important libraries

In [ ]:
import re, math
import nltk
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
# import tensorflow as tf
import matplotlib.pyplot as plt         
# from wordcloud import WordCloud
import os
import heapq
from nltk.stem.isri import ISRIStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# loading data

In [ ]:
                                                                                                      !wget https://liquidtelecom.dl.sourceforge.net/project/easc-corpus/EASC/EASC.zip

--2022-01-06 10:20:56--  https://liquidtelecom.dl.sourceforge.net/project/easc-corpus/EASC/EASC.zip
Resolving liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)... 197.155.77.8
Connecting to liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)|197.155.77.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1519413 (1.4M) [application/octet-stream]
Saving to: ‘EASC.zip’

EASC.zip            100%[===================>]   1.45M  1.24MB/s    in 1.2s    

2022-01-06 10:20:59 (1.24 MB/s) - ‘EASC.zip’ saved [1519413/1519413]



In [ ]:
!unzip EASC.zip

Archive:  EASC.zip
  inflating: EASC/COPYRIGHT          
   creating: EASC/EASC-ISO-Arabic/
   creating: EASC/EASC-ISO-Arabic/Articles/
   creating: EASC/EASC-ISO-Arabic/Articles/Topic1/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic1/Art and Music (1).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic10/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic10/Art and Music (10).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic100/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic100/politics (16).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic101/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic101/politics (17).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic102/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic102/politics (18).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic103/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic103/politics (19).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic104/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic104/politi

In [ ]:
!ls

EASC  EASC.zip	sample_data


In [ ]:
cd EASC

/content/EASC


In [ ]:
ls

COPYRIGHT  EASC-ISO-Arabic/  EASC-UTF-8/  README  SOURCES.CSV


In [ ]:
!wget https://github.com/mohataher/arabic-stop-words/blob/master/list.txt

--2022-01-06 10:21:00--  https://github.com/mohataher/arabic-stop-words/blob/master/list.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘list.txt’

list.txt                [ <=>                ] 321.50K  --.-KB/s    in 0.05s   

2022-01-06 10:21:00 (6.10 MB/s) - ‘list.txt’ saved [329218]



In [ ]:
!cat EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.1.A 
!cat EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.2.B
!cd  EASC/EASC-UTF-8/MTurk/Topic1/
!find -maxdepth 1 -type f -ls -exec file -b {} \;
!pwd


cat: EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.1.A: No such file or directory
cat: EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.2.B: No such file or directory
/bin/bash: line 0: cd: EASC/EASC-UTF-8/MTurk/Topic1/: No such file or directory
  6031925      4 -rw-r--r--   1 root     root         2718 Feb 20  2010 ./COPYRIGHT
find: ‘file’: No such file or directory
  6034396    324 -rw-r--r--   1 root     root       329218 Jan  6 10:21 ./list.txt
find: ‘file’: No such file or directory
  6034391     16 -rw-r--r--   1 root     root        13078 Feb 20  2010 ./SOURCES.CSV
find: ‘file’: No such file or directory
  6034390      4 -rw-r--r--   1 root     root         2317 Feb 20  2010 ./README
find: ‘file’: No such file or directory
/content/EASC


In [ ]:

# Build the Dataset
train_dir_name = '/content/EASC/EASC-UTF-8/Articles/'
test_dir_name = '/content/EASC/EASC-UTF-8/MTurk/'
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry) 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles
train_files_path = getListOfFiles(train_dir_name)
train_files_path[:15]
len(train_files_path)
test_files_path = getListOfFiles(test_dir_name)
test_files_path[:15]
len(test_files_path)

len(train_files_path) * 5
data = pd.DataFrame(columns = ['Orignal' ,'Summary1' ,'Summary2' ,'Summary3' ,'Summary4' ,'Summary5'])
j = 0
for i in range(len(train_files_path)) :
  train_path = train_files_path[i]
  test_path = test_files_path[j : j + 5]
  j+=5
  
  train_file = open(train_path , 'r' ,encoding = 'utf-8').read()
  sum1 =  open(test_path[0] , 'r' ,encoding = 'utf-8').read()
  sum2 =  open(test_path[1] , 'r' ,encoding = 'utf-8').read()
  sum3 =  open(test_path[2] , 'r' ,encoding = 'utf-8').read()
  sum4 =  open(test_path[3] , 'r' ,encoding = 'utf-8').read()
  sum5 =  open(test_path[4] , 'r' ,encoding = 'utf-8').read()
  
  row_list = []
  row_list.extend((train_file, sum1, sum2,sum3,sum4,sum5))
  data.loc[len(data)] = row_list

  
  
  
  
text = data.iloc[1]['Orignal']
data


,Orignal,Summary1,Summary2,Summary3,Summary4,Summary5
0,﻿السياسة البربرية تعد هذه السياسة من بين النقط...,ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاس...,ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاس...,ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاس...,فجمع حوله ثلة من المفكرين ذوي هذا الاتجاه وخاص...,السياسة البربرية تعد هذه السياسة من بين النقط ...
1,﻿الانسان الآلي (أو الروبوت) (بالإنجليزية: Robo...,الانسان الآلي (او الروبوت) (بالانجليزية: Robot...,الانسان الآلي (او الروبوت) (بالانجليزية: Robot...,الانسان الآلي (او الروبوت) (بالانجليزية: Robot...,الانسان الآلي (او الروبوت) (بالانجليزية: Robot...,وضع الكثير من الدراسات و التوقعات عن هذا الانس...
2,﻿تقع جمهورية الصين الشعبية في الجزء الشرقي من ...,تقع جمهورية الصين الشعبية في الجزء الشرقي من ق...,تقع جمهورية الصين الشعبية في الجزء الشرقي من ق...,تبدا حدود الصين في اقصي الشمال من الخط المركزي...,﻿ اراضي سواحلها منبسطة وعلى هذه السواحل موانئ ...,تقع جمهورية الصين الشعبية في الجزء الشرقي من ق...
3,﻿تعتبر القدس مدينة مقدسة بالنسبة للديانات التو...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...,اول اسم ثابت لمدينة القدس هو اورسالم قبل خمسة ...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...,وتذكر في تاريخ اليهود والمسيحيين باعتبارها ال...
4,﻿ كشف تقرير لمصلحة الإحصاءات العامة عن استقرار...,كشف تقرير لمصلحة الاحصاءات العامة عن استقرار ...,كشف تقرير لمصلحة الاحصاءات العامة عن استقرار ...,كشف تقرير لمصلحة الاحصاءات العامة عن استقرار ...,كشف تقرير لمصلحة الاحصاءات العامة عن استقرار ...,كشف تقرير لمصلحة الاحصاءات العامة عن استقرار ...
...,...,...,...,...,...,...
148,﻿الكبد أكبر عضو غددي في الجسم يزن حوالي كيلو و...,ويقع في الجانب الايمن من التجويف البطني تحت ا...,الكبد اكبر عضو غددي في الجسم يزن حوالي كيلو ون...,الكبد اكبر عضو غددي في الجسم يزن حوالي كيلو ون...,الكبد اكبر عضو غددي في الجسم يزن حوالي كيلو ون...,الكبد اكبر عضو غددي في الجسم يزن حوالي كيلو ون...
149,﻿كانت إيطاليا آخر الدول الأوروبية التي دخلت مج...,كانت ايطاليا آخر الدول الاوروبية التي دخلت مجا...,بدات ايطاليا العزم على احتلال ليبيا فقامت بفتح...,وفى يوم 28 سبتمبر 1911 اقبل الموعد المحدد لان...,كانت ايطاليا آخر الدول الاوروبية التي دخلت مجا...,كانت ايطاليا آخر الدول الاوروبية التي دخلت مجا...
150,﻿ قالت مصادر في حركة فتح كبرى الفصائل الفلسطين...,وكان وزير شؤون الاسرى سفيان ابو زايدة ومعه ال...,قالت مصادر في حركة فتح كبرى الفصائل الفلسطيني...,قالت مصادر في حركة فتح كبرى الفصائل الفلسطيني...,قالت مصادر في حركة فتح كبرى الفصائل الفلسطيني...,"وبحسب هذه المصادر فان ""قادة الحركة في قطاع غز..."
151,﻿ اعتقل إندونيسي من قبل الشرطة بعد أن أثار جدل...,اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدلا...,اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدلا...,﻿ اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدل...,﻿ اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدل...,اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدلا...


data evalute

In [ ]:

i = 1
sentences_scores = {}
wordsCounter = 0.0
for col in range(1,6):
  sentences_tokens = nltk.sent_tokenize(data[f'Summary{col}'][i])
  for sentence in sentences_tokens:
    if sentence not in sentences_scores.keys():
      sentences_scores[sentence] = 1 
    else:
        sentences_scores[sentence] += 1
l = [str(k) for k,v in sentences_scores.items() if v >= 3]
refrenc_summary=' '.join(l) 
refrenc_summary

'الانسان الآلي (او الروبوت) (بالانجليزية: Robot) عبارة عن اداة ميكانيكية قادرة على القيام بفعاليات مبرمجة سلفا ويقوم الانسان الآلي بانجاز تلك الفاليات اما بايعاز وسيطرة مباشرة من الانسان او بايعاز من برامج حاسوبية والفعاليات التي تبرمج الانسان على اداءها ع. وضع الكثير من الدراسات و التوقعات عن هذا الانسان الآلي التي فشلت فيما بعد . و اليوم و بسبب التطور الهائل للحواسيب و الذكاء الاصطناعي و التقنيات و الهوس في تطوير البرامج الفضائية فنحن على حافة انجاز كبير آخر في مجال علوم تصميم الانسالات . هناك انواع عديدة من الانسان الآلي منها ما يستعمل في صناعي والتي هي عبارة عن اجهزة اتوماتيكية يمكن تطويعها و اعادة برمجتها ويستعمل لاغراض عديدة بامكانها الحركة على ثلاثة محاور او اكثر ويستعمل هذا النوع في الشركات الصناعية الكبرى لغرض لحم المعادن و الصباغة . وهناك منه ما هو قادر على اعادة تجميع نفسه بصورة شبه مستقلة على سبيل المثال تصغير حجمه للمرور خلال نفق ضيق وهذه الانسالةات تحوي في نموذجها على عدة روابط مع وحدة المعالجة المركزية و ومستقبلات الايعزات و الذاكرة وهذه الانسالةات قادرة على بعض الحركات 

In [ ]:
data.iloc[1]['Orignal']

'\ufeffالانسان الآلي (أو الروبوت) (بالإنجليزية: Robot) عبارة عن أداة ميكانيكية قادرة على القيام بفعاليات مبرمجة سلفا ويقوم الانسان الآلي بإنجاز تلك الفاليات إما بإيعاز وسيطرة مباشرة من الإنسان او بإيعاز من برامج حاسوبية والفعاليات التي تبرمج الانسان على أداءها عادة تكون فعاليات شاقة او خطيرة مثل البحث عن الألغام والفضاء الخارجي وتنظيف الفضلات الناتجة في المفاعلات النووية . تم تقديم كلمة انسانة لأول مرة في مسرحية الكاتب المسرحي التشيكي كارل كابك عام 1920 .و كان عنوان المسرحية وقتها رجال آليون عالميون . وهي تعني في اللغة التشيكية تعني العمل الشاق رغم أن كارل هو أول من استعمل هذه الكلمة، لكن ليس من اخترعها، بل أخوه جوزيف الذي اشتقها مساعدة منه لأخيه من الكلمة التشيكية "Robota" والتي تعني السُخرة أو العمل الجبري . من هذا التاريخ بدأت هذه الكلمة تنتشر في الكتب و أفلام الخيال العلمي الأولى التي أعطت فكرة و تصور علمي عن هؤلاء الرجال الآليون الذين سيغزون العالم. و أعطت أفق كبير ووعود عظيمة للإنسان الأعجوبة الذي سيتدخل في أمور كثيرة و أهمها الصناعة .وضع الكثير من الدراسات و التوقعات عن هذا الإن

#Prapere The Comparison Dictionary

In [ ]:
model_comparison_table = {}

In [ ]:
model_comparison_table['model_name'] = []
model_comparison_table['recall'] = []
model_comparison_table['prceision'] = []
model_comparison_table['score-f1'] = []

In [ ]:
def add_to_dictionary(model_name,score):  
  """add detaisl to dictionary"""

  model_comparison_table['model_name'].append(model_name)
  model_comparison_table['recall'].append(score['rouge-1']['r'])
  model_comparison_table['prceision'].append(score['rouge-1']['p'])
  model_comparison_table['score-f1'].append(score['rouge-1']['f'])
    


# Proprecess

## Remove url and mintion and hashtags

In [ ]:
def remove_url(tweet):

    # remove URL
    tweet = re.sub('https?\S+', ' ', tweet)
    
    # Remove usernames
    tweet = re.sub(r"@\S+",'',tweet)
    
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    tweet = re.sub('^[\u0621-\u064A\u0660-\u0669 ]+$', ' ', tweet)
    tweet = re.sub('\.+', ' ', tweet)
    tweet = re.sub(':', '', tweet)
    tweet = re.sub('!', '', tweet)
    tweet = re.sub('،','',tweet)
    tweet = re.sub('-',' ',tweet)
    tweet = re.sub('_',' ',tweet)
    tweet = re.sub('\*',' ',tweet)
    tweet = re.sub('"',' ',tweet)
      # remove extra whitespace
    tweet = re.sub('\s+', ' ', tweet) 
    
   
    
    
    return tweet

## Remove Stopwords

In [ ]:
def remove_stepwords(tweet):
    stop_words=nltk.corpus.stopwords.words("arabic")
#     print(stop_words)
    t = " ".join([w for w in str(tweet).split() if not w in stop_words])
    return t

Arabic ProPrecess

In [ ]:
pip install PyArabic

     |████████████████████████████████| 126 kB 5.2 MB/s 


In [ ]:
import pyarabic.araby as pa
import string
import re
def proprcess_arbic(text):
  # حذف الحركات
  text = pa.strip_tashkeel(text) 
  # حذف الزخرفات
  text = pa.strip_diacritics(text)
   # حذف التطويل
  text = pa.strip_tatweel(text)
    

    #remove longation
  text = re.sub("[إأآا]", "ا", text)
  text = re.sub("ى", "ي", text)
  text = re.sub("ؤ", "ء", text)
  text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", text)
    
    
    # remove URL
  text = re.sub('http\S+\s*', ' ', text)
    
    # Remove usernames
  text = re.sub(r"@[^\s]+[\s]?",'',text)
    
    # remove special characters 
  text = re.sub("@[ا-ي0-9]+", " ", text)
  text = re.sub("[أ-ي]#+", " ", text)
  text = re.sub("#[أ-ي]+", " ", text)

  #remove no_arabic
  text = ' '.join(pa.tokenize(text, conditions=pa.is_arabicrange))
      
    
    # # remove Numbers
    # tweet = re.sub('^[\u0621-\u064A\u0660-\u0669 ]+$', '', tweet)
    # tweet = re.sub('\.+', '', tweet)
    # tweet = re.sub(':', '', tweet)
    # tweet = re.sub('!', '', tweet)
    # tweet = re.sub('،','',tweet)
    # tweet = re.sub('-','',tweet)
    # tweet = re.sub('_','',tweet)
    # return tweet
  
  return text

proprcess_arbic(text)

'\ufeff الانسان الالي او الروبوت بالانجليزيه عباره عن اداه ميكانيكيه قادره علي القيام بفعاليات مبرمجه سلفا ويقوم الانسان الالي بانجاز تلك الفاليات اما بايعاز وسيطره مباشره من الانسان او بايعاز من برامج حاسوبيه والفعاليات التي تبرمج الانسان علي اداءها عاده تكون فعاليات شاقه او خطيره مثل البحث عن الالغام والفضاء الخارجي وتنظيف الفضلات الناتجه في المفاعلات النوويه تم تقديم كلمه انسانه لاول مره في مسرحيه الكاتب المسرحي التشيكي كارل كابك عام و كان عنوان المسرحيه وقتها رجال اليون عالميون وهي تعني في اللغه التشيكيه تعني العمل الشاق رغم ان كارل هو اول من استعمل هذه الكلمه ، لكن ليس من اخترعها ، بل اخوه جوزيف الذي اشتقها مساعده منه لاخيه من الكلمه التشيكيه والتي تعني السخره او العمل الجبري من هذا التاريخ بدات هذه الكلمه تنتشر في الكتب و افلام الخيال العلمي الاولي التي اعطت فكره و تصور علمي عن هءلاء الرجال الاليون الذين سيغزون العالم و اعطت افق كبير ووعود عظيمه للانسان الاعجوبه الذي سيتدخل في امور كثيره و اهمها الصناعه وضع الكثير من الدراسات و التوقعات عن هذا الانسان الالي التي فشلت فيما بعد و ل

## Text Proprecess

In [ ]:
def proprecess(text):
    
    text = remove_url(text)
    text = remove_stepwords(text)
    text = proprcess_arbic(text)
    
    return text   

# Enter TexT

In [ ]:
text = """في عصر البيانات الضخمة، هناك انفجار في كمية البيانات النصية الصادرة من  مصادر متنوعة، هذا الحجم من النص هو كنز لا يقدر بثمن للمعلومات لذلك لابد من تلخيص فعال لاستخراج المعلومات المفيدة، نظرًا لأن التلخيص اليدوي للنص يعد مكلفًا للوقت ومهمة شاقة بشكل عام، فإن أتمتة المهمة تكتسب شعبية متزايدة وبالتالي تشكل دافعًا قويًا للبحث الأكاديمي.
هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.
التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.
تجري عملية التلخيص التلقائي للنص بواسطة تقنيتين أساسيتين هما: التلخيص الاستخراجي والتخليص التجريدي.
التلخيص الاستخراجي يلتقط الجمل مباشرة من المستند بناءً على أهميتها لتشكيل ملخص جديد، تعمل هذه الطريقة من خلال تحديد أقسام النص المهمة وتجميعها معًا لإنتاج نسخة مختصرة، وبالتالي فهي تعتمد فقط على استخراج الجمل من النص الأصلي.
 ركزت معظم أبحاث التلخيص اليوم على التلخيص الاستخراجي، لأنه أسهل ويعطي ملخصات نحوية طبيعية ومقبولة.
وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة.
"""

In [ ]:
print(text)

في عصر البيانات الضخمة، هناك انفجار في كمية البيانات النصية الصادرة من  مصادر متنوعة، هذا الحجم من النص هو كنز لا يقدر بثمن للمعلومات لذلك لابد من تلخيص فعال لاستخراج المعلومات المفيدة، نظرًا لأن التلخيص اليدوي للنص يعد مكلفًا للوقت ومهمة شاقة بشكل عام، فإن أتمتة المهمة تكتسب شعبية متزايدة وبالتالي تشكل دافعًا قويًا للبحث الأكاديمي.
هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.
التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.
تجري عملية التلخيص التلقائي للنص بواسط

In [ ]:

sentences_tokens = nltk.sent_tokenize(text)
sentences_tokens
    


['في عصر البيانات الضخمة، هناك انفجار في كمية البيانات النصية الصادرة من  مصادر متنوعة، هذا الحجم من النص هو كنز لا يقدر بثمن للمعلومات لذلك لابد من تلخيص فعال لاستخراج المعلومات المفيدة، نظرًا لأن التلخيص اليدوي للنص يعد مكلفًا للوقت ومهمة شاقة بشكل عام، فإن أتمتة المهمة تكتسب شعبية متزايدة وبالتالي تشكل دافعًا قويًا للبحث الأكاديمي.',
 'هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.',
 'التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.',
 'تجري عملية التلخيص التلق

In [ ]:

data_clean =[]

for i in sentences_tokens:
  clean = proprecess(i)
  
  words_tokens = nltk.word_tokenize(clean)
  for j in words_tokens:
    data_clean.append(j)
len(data_clean)



232

In [ ]:
st = ISRIStemmer()
words_stemm = [st.stem(word) for word in words_tokens]
len(words_stemm)

45

# get summariization


## word frequencies 

In [ ]:

word_frequencies = {}
for word in words_stemm:
    if word not in word_frequencies.keys():
        word_frequencies[word] = 1
    else:
        word_frequencies[word] += 1

maximum_frequency_word = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequency_word)

In [ ]:
  
#  Calculate Each Sentence Frequency from the Term Frequencies 

sentences_scores = {}
wordsCounter = 0.0
for sentence in sentences_tokens:
    for word in nltk.word_tokenize(sentence):
        word = st.stem(word)
        if word in word_frequencies.keys():
            wordsCounter += 1;
            if sentence not in sentences_scores.keys():
                sentences_scores[sentence] = word_frequencies[word]
            else:
                sentences_scores[sentence] += word_frequencies[word]
    
    wordsCounter = 0

#Get summary with only highest top 3 
   
summary = heapq.nlargest(3, sentences_scores, key=sentences_scores.get)
summary = [str(i) for i in sentences_tokens if i in summary]
summary = ''.join(summary).replace("\ufeff","")
print(summary)

    


هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة.


In [ ]:
print(summary)

هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة.


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 
# summary = ''.join(summary).replace("\ufeff","")
# print(summary)
# help(rouge.get_scores)
rouge = Rouge()
# scores = rouge.get_scores(summary, refrenc_summary)

scores = rouge.get_scores(hyps=summary, refs=refrenc_summary, avg=True)
add_to_dictionary('word frequency',pd.DataFrame(scores))
print(pd.DataFrame(scores))

    rouge-1  rouge-2   rouge-l
r  0.053691      0.0  0.053691
p  0.062500      0.0  0.062500
f  0.057762      0.0  0.057762


In [ ]:
print(model_comparison_table)

{'model_name': ['word frequency'], 'recall': [0.053691275167785234], 'prceision': [0.0625], 'score-f1': [0.05776172788072349]}


## K_mean

In [ ]:
! wget https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_100_wiki.zip

--2022-01-06 10:21:09--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_100_wiki.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 507605088 (484M) [application/zip]
Saving to: ‘full_grams_sg_100_wiki.zip’

full_grams_sg_100_w 100%[===================>] 484.09M   137MB/s    in 3.7s    

2022-01-06 10:21:13 (131 MB/s) - ‘full_grams_sg_100_wiki.zip’ saved [507605088/507605088]



In [ ]:
! unzip full_grams_sg_100_wiki.zip

Archive:  full_grams_sg_100_wiki.zip
  inflating: full_grams_sg_100_wiki.mdl  
  inflating: full_grams_sg_100_wiki.mdl.trainables.syn1neg.npy  
  inflating: full_grams_sg_100_wiki.mdl.wv.vectors.npy  


In [ ]:
pip install gensim

In [ ]:
import gensim
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn import cluster
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans

#Get the pretrained model
t_model = gensim.models.Word2Vec.load('full_grams_sg_100_wiki.mdl')

data_clean =[]
sentences_tokens = nltk.sent_tokenize(text)

senCount=0;
sentenceVec = {}
tsne = TSNE(n_components=2)
for sen in sentences_tokens:
    sen = [st.stem(word) for word in nltk.word_tokenize(sen) if st.stem(word) in t_model.wv.vocab]
    if len(sen) >= 1:
        sentenceVec[senCount]=np.mean(t_model.wv[sen], axis=0)
        senCount+=1
 
    
NUM_CLUSTERS=3

newSenVec = []
for i in sentenceVec:
    newSenVec.append(sentenceVec[i])


kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0)

kmeans = kmeans.fit(newSenVec)
avg2 = []
closest2 = []
for j in range(NUM_CLUSTERS):
    idx = np.where(kmeans.labels_ == j)[0]
    
    avg2.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,newSenVec)
ordering = sorted(range(NUM_CLUSTERS), key=lambda k: avg2[k])
summary2 = ' '.join([sentences_tokens[closest[idx]] for idx in ordering])

print(summary2)


وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة. تجري عملية التلخيص التلقائي للنص بواسطة تقنيتين أساسيتين هما: التلخيص الاستخراجي والتخليص التجريدي. ركزت معظم أبحاث التلخيص اليوم على التلخيص الاستخراجي، لأنه أسهل ويعطي ملخصات نحوية طبيعية ومقبولة.


In [ ]:
from rouge import Rouge 
summary2 = ''.join(summary2).replace("\ufeff","")
# print(summary)
# help(rouge.get_scores)
rouge = Rouge()
# scores = rouge.get_scores(summary, refrenc_summary)

scores = rouge.get_scores(hyps=summary2, refs=refrenc_summary, avg=True)
add_to_dictionary('K_mean based on the closet to the center  ',pd.DataFrame(scores))
print(pd.DataFrame(scores))

    rouge-1  rouge-2   rouge-l
r  0.033557      0.0  0.033557
p  0.068493      0.0  0.068493
f  0.045045      0.0  0.045045


In [ ]:
print(summary2)

وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة. تجري عملية التلخيص التلقائي للنص بواسطة تقنيتين أساسيتين هما: التلخيص الاستخراجي والتخليص التجريدي. ركزت معظم أبحاث التلخيص اليوم على التلخيص الاستخراجي، لأنه أسهل ويعطي ملخصات نحوية طبيعية ومقبولة.


# TestRank_Text_Summarization

In [ ]:
 
data_clean =[]
sentences = nltk.sent_tokenize(text)


senCount=0;
sentence_vectors = []
sentenceVec = {}
tsne = TSNE(n_components=2)
for sen in sentences_tokens:
    sen = [st.stem(word) for word in nltk.word_tokenize(sen) if st.stem(word) in t_model.wv.vocab]
    if len(sen) >= 1:
        sentenceVec[senCount]=np.mean(t_model.wv[sen], axis=0)
        sentence_vectors.append(sentenceVec[senCount])
        senCount+=1
 
print(len(sentence_vectors[0]))

# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Specify number of sentences to form the summary
sn = 3
l =[]
# Generate summary
for i in range(sn):
  l.append(ranked_sentences[i][1])
summary = []
for i in sentences:
  if i in l:
    summary.append(i)
summary


100


['هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.',
 'التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.',
 'وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة.']

In [ ]:
from rouge import Rouge 
summary = ''.join(summary).replace("\ufeff","")

rouge = Rouge()
# scores = rouge.get_scores(summary, refrenc_summary)

scores = rouge.get_scores(hyps=summary, refs=refrenc_summary, avg=True)
add_to_dictionary('PageRank Algorithom',pd.DataFrame(scores))
print(summary)

هناك تطبيقات مهمة لتلخيص النص في مختلف المهام المتعلقة بالبرمجة اللغوية العصبية مثل تصنيف النص، والإجابة على الأسئلة، وتلخيص النصوص القانونية، وتلخيص الأخبار، وتوليد العناوين، ويمكن دمج إنشاء الملخصات في هذه الأنظمة كمرحلة وسيطة تساعد على تقليل طول المستند.التلخيص التلقائي للنص هو مهمة إنتاج ملخص موجز دون أي مساعدة بشرية مع الحفاظ على معنى المستند النصي الأصلي، إنه أمر صعب للغاية، لأننا عندما نلخص كبشر جزءًا من النص، فإننا نقرأه بالكامل عادةً لتطوير فهمنا، ثم نكتب ملخصًا يسلط الضوء على نقاطه الرئيسية، ونظرًا لأن أجهزة الحاسوب تفتقر إلى المعرفة البشرية والقدرة اللغوية، فإنها تجعل التلخيص التلقائي للنص مهمة صعبة للغاية.وتهدف طرق التلخيص التجريدية إلى إنتاج ملخص عن طريق تفسير النص باستخدام تقنيات معالجة اللغة الطبيعية المتقدمة من أجل إنشاء نص أقصر جديد، قد لا تظهر أجزاء منه كجزء من المستند الأصلي، والذي ينقل المعلومات الأكثر أهمية من النص الأصلي، مما يتطلب إعادة صياغة الجمل ودمج المعلومات من النص الكامل لتوليد الملخصات مثل الملخصات المكتوبة بواسطة الإنسان عادة.


# The Result

In [ ]:
df = pd.DataFrame(model_comparison_table)
df

,model_name,recall,prceision,score-f1
0,word frequency,0.053691,0.062500,0.057762
1,K_mean based on the closet to the center,0.033557,0.068493,0.045045
2,PageRank Algorithom,0.053691,0.062500,0.057762
